In [5]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
import os
from datetime import datetime
import locale

# Configuração de Locale para formatação em reais
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # Para sistemas UNIX
# Para Windows, você pode precisar usar 'Portuguese_Brazil.1252'
# locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil.1252')

# Função para carregar a planilha local e enviar para o Google Sheets
def atualizar_google_sheets(local_planilha, aba_google_sheets, intervalo):
    try:
        # Ler a planilha local (arquivo Excel ou CSV)
        if local_planilha.endswith('.xlsx'):
            df = pd.read_excel(local_planilha)
        elif local_planilha.endswith('.csv'):
            df = pd.read_csv(local_planilha)
        else:
            raise ValueError("Formato de arquivo não suportado. Use .xlsx ou .csv")

        # Substituir valores NaN por string vazia ou outro valor apropriado
        df = df.fillna('')

        # Convertendo DataFrame para lista de listas (primeira linha são os nomes das colunas)
        dados = [df.columns.values.tolist()] + df.values.tolist()

        # Atualizar a aba da planilha com os dados
        aba_google_sheets.update(intervalo, dados)  # Corrigi para usar a aba diretamente
        print("Google Sheets atualizado com sucesso!")
    except Exception as e:
        print(f"Erro ao carregar a planilha local ou atualizar o Google Sheets: {e}")

# Configuração de Autenticação com a API do Google
credenciais_arquivo = r"C:\Users\AmandaAlves\CP SRN Dropbox\AMANDA - TI\Arquivos Amanda\TI\CURSOS\AMANDA\atualizar-437920-96d927f6c56c.json"

# Escopos necessários
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Autenticação usando o arquivo de credenciais
try:
    credenciais = Credentials.from_service_account_file(credenciais_arquivo, scopes=scopes)
    gc = gspread.authorize(credenciais)
    print("Autenticação bem-sucedida!")
except Exception as e:
    print(f"Erro na autenticação: {e}")
    exit()

# Nome da planilha no Google Sheets
nome_planilha_google = 'testepedido ciclo 14 - atualizar'

# Conectar à planilha do Google Sheets e à primeira aba
try:
    planilha = gc.open(nome_planilha_google)
    aba_google_sheets = planilha.sheet1  # Acessa a primeira aba automaticamente
    print(f"Conectado à planilha: {nome_planilha_google}")
except Exception as e:
    print(f"Erro ao conectar à planilha: {e}")
    exit()

# Função para encontrar o arquivo mais recente no padrão 'ConsultaPedidos_*.xlsx'
def encontrar_arquivo_mais_recente():
    pasta_downloads = os.path.expanduser('~/Downloads')
    arquivos = [os.path.join(pasta_downloads, f) 
                for f in os.listdir(pasta_downloads) 
                if f.startswith('ConsultaPedidos_') and f.endswith('.xlsx')]
    
    if arquivos:
        return max(arquivos, key=os.path.getmtime)
    else:
        return None

# Obter o arquivo mais recente e atualizar o Google Sheets
arquivo_mais_recente = encontrar_arquivo_mais_recente()
if arquivo_mais_recente:
    # Ler o arquivo Excel e adicionar a data de extração
    df = pd.read_excel(arquivo_mais_recente)
    df['data_extracao'] = datetime.now().date()
    
    # Atualizar o Google Sheets
    intervalo = 'A1'  # Substitua pelo intervalo apropriado na planilha do Google Sheets
    atualizar_google_sheets(arquivo_mais_recente, aba_google_sheets, intervalo)
else:
    print("Nenhum arquivo correspondente encontrado na pasta de downloads.")


Autenticação bem-sucedida!
Conectado à planilha: testepedido ciclo 14 - atualizar


C:\Users\AmandaAlves\AppData\Local\Temp\ipykernel_9000\2588548212.py:31: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  aba_google_sheets.update(intervalo, dados)  # Corrigi para usar a aba diretamente


Google Sheets atualizado com sucesso!


In [6]:
df

,CodigoPedido,Cód Externo Pedido,SituaçãoFiscal,NotaFiscal,Pessoa,NomePessoa,Papel,Ordem de Pedido,Pontos,QtdeItens,...,CodCD,CanalDistribuicao,Presente,Detalhe Entrega,Nome do Recebedor,DetalheMeioCaptacao,Código OMS,Link Rastreio,CategoriaDispositivo,data_extracao
0,413333107,139383896BOT-F1,NF Emitida,10914.0,19034640,LAYZA SOUSA DE ANIS,Consumidor Final,1,137,3,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,Layza Sousa de Anis,NaN,NaN,NaN,Desktop,2024-10-29
1,413334419,NaN,NF Emitida,354864.0,18798080,CINTHYAN FÁTIMA DE OLIVEIRA ORDONIS,Bronze,7,416,22,...,740,13863 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,portalreBotiStargate,NaN,NaN,Desktop,2024-10-29
2,413334560,NaN,NF Emitida,10600.0,11915631,LUCIANA ANDREIA DA SILVA,Ouro,55,223,3,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,apprebotiflutter,NaN,NaN,Desktop,2024-10-29
3,413335224,139384448BOT-F1,NF Emitida,10915.0,19034808,RENATO MARINO,Consumidor Final,1,631,4,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,Renato Marino,NaN,NaN,NaN,Desktop,2024-10-29
4,413336114,NaN,NF Emitida,46790.0,80524,ANDREA PAZZINI LIMA,Platina,136,212,5,...,1171,20213 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,apprebotiflutter,NaN,NaN,Desktop,2024-10-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3155,415680487,NaN,Não Faturado,NaN,8370629,MELLYSSA QUERINO COSTA,Bronze,3,412,5,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,NaN,NaN,NaN,Desktop,2024-10-29
3156,415680727,NaN,Não Faturado,NaN,4117958,FABIANA C FERREIRA,Ouro,5,556,5,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,apprebotiflutter,NaN,NaN,Desktop,2024-10-29
3157,415681339,139798578QDB-F1,Não Faturado,NaN,17420271,BEATRIZ SOLETTI PEREIRA,Consumidor Final,2,89,3,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,Beatriz Pereira,NaN,NaN,NaN,Desktop,2024-10-29
3158,415682879,NaN,Não Faturado,NaN,14403135,MARIA BEATRIZ DA SILVA,Bronze,2,0,0,...,1519,23237 - GHN DISTRIBUIDORA DE PRODUTOS DE BELEZ...,NaN,NaN,NaN,NaN,NaN,NaN,Desktop,2024-10-29


In [9]:
# Calcular a soma total de 'ValorPraticado' por 'Estrutura'
soma_por_estrutura = df.groupby('Estrutura')['ValorPraticado'].sum().reset_index()
soma_por_estrutura

,Estrutura,ValorPraticado
0,BASE IR,21393.05
1,CONSUMIDOR,87496.31
2,DESATIVADOS,5578.90
3,DIAMANTE,1039348.56
4,ESMERALDA,32242.17
5,FVC - 13863 - A - SRN 1,194468.11
6,FVC - 13863 - A - SRN 2,182111.08
7,FVC - 13863 - SRN KACC 1,88430.96
8,FVC - 20213 - A - SRN 1,83620.63
9,FVC - A - 13863 - SRN 3,185650.25


In [10]:
# Função para formatar valores em reais
def formatar_reais(valor):
    return locale.currency(valor, grouping=True)
# Formatar os valores em reais
soma_por_estrutura['SomaValorPraticado'] = soma_por_estrutura['ValorPraticado'].apply(formatar_reais)

# Exibir o resultado
display(soma_por_estrutura[['Estrutura', 'SomaValorPraticado']])

,Estrutura,SomaValorPraticado
0,BASE IR,"R$ 21.393,05"
1,CONSUMIDOR,"R$ 87.496,31"
2,DESATIVADOS,"R$ 5.578,90"
3,DIAMANTE,"R$ 1.039.348,56"
4,ESMERALDA,"R$ 32.242,17"
5,FVC - 13863 - A - SRN 1,"R$ 194.468,11"
6,FVC - 13863 - A - SRN 2,"R$ 182.111,08"
7,FVC - 13863 - SRN KACC 1,"R$ 88.430,96"
8,FVC - 20213 - A - SRN 1,"R$ 83.620,63"
9,FVC - A - 13863 - SRN 3,"R$ 185.650,25"
